In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class TrainingConfig:
    root_dir: str
    trained_model_path: str
    learning_rate: float
    weight_decay: float
    num_epochs: int
    patience: int
    num_layers_to_unfreeze: int
    pretrained_model_path: str



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [18]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
# from cnnClassifier.entity import DataPreprocessingConfig
from cnnClassifier.entity.config_entity import PrepareBaseModelConfig
from pathlib import Path

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training_cfg = self.config.training
        pretrained_model_path = self.config.prepare_base_model.updated_base_model_path
        create_directories([training_cfg.root_dir])
        return TrainingConfig(
            root_dir=training_cfg.root_dir,
            trained_model_path=training_cfg.trained_model_path,
            pretrained_model_path=pretrained_model_path,
            learning_rate=0.0001,          # you can push this to yaml later
            weight_decay=1e-4,
            num_epochs=5,
            patience=5,
            num_layers_to_unfreeze=30
        )

In [ ]:
config = read_yaml(CONFIG_FILE_PATH)
confiig=config.prepare_base_model.updated_base_model_path
traincnfig=config.training
print(traincnfig.trained_model_path)
print(confiig)


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim

class TrainModel:
    def __init__(self, config: TrainingConfig, model: nn.Module,
                 train_loader, val_loader, device="cuda"):
        self.config = config
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Load pretrained (updated) weights if available
        if self.config.pretrained_model_path and Path(self.config.pretrained_model_path).exists():
            print(f"📥 Loading pretrained weights from {self.config.pretrained_model_path}")
            self.model.load_state_dict(torch.load(self.config.pretrained_model_path, map_location=device))

        # Loss, optimizer, scheduler
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(
            self.model.parameters(),
            lr=self.config.learning_rate,
            weight_decay=self.config.weight_decay
        )
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)

    def train(self):
        print(f"\n🚀 Starting Training for {self.config.num_epochs} epochs")
        print(f"   - Learning rate: {self.config.learning_rate}")
        print(f"   - Weight decay: {self.config.weight_decay}")
        print(f"   - Patience: {self.config.patience}")
        print(f"   - Unfreezing last {self.config.num_layers_to_unfreeze} layers")

        best_acc = 0.0
        patience_counter = 0

        for epoch in range(self.config.num_epochs):
            # 🔹 Training
            self.model.train()
            running_loss, correct_train, total_train = 0.0, 0, 0

            for images, labels in self.train_loader:
                images, labels = images.to(self.device), labels.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                correct_train += (preds == labels).sum().item()
                total_train += labels.size(0)

            train_acc = 100 * correct_train / total_train
            epoch_loss = running_loss / len(self.train_loader.dataset)

            # 🔹 Validation
            self.model.eval()
            correct_val, total_val = 0, 0
            with torch.no_grad():
                for images, labels in self.val_loader:
                    images, labels = images.to(self.device), labels.to(self.device)
                    outputs = self.model(images)
                    _, preds = torch.max(outputs, 1)
                    correct_val += (preds == labels).sum().item()
                    total_val += labels.size(0)

            val_acc = 100 * correct_val / total_val

            print(f"📊 Epoch {epoch+1}/{self.config.num_epochs} "
                  f"| Loss: {epoch_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

            # 🔹 Save best model
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(self.model.state_dict(), self.config.trained_model_path)
                print(f"   💾 Best model updated @ {val_acc:.2f}% "
                      f"-> saved to {self.config.trained_model_path}")
                patience_counter = 0
            else:
                patience_counter += 1

            # 🔹 Early stopping
            if patience_counter >= self.config.patience:
                print("⏹️ Early stopping triggered")
                break

            self.scheduler.step()

        print(f"✅ Training completed! Best Val Accuracy: {best_acc:.2f}%")
        return best_acc


In [ ]:
config = ConfigurationManager()
training_config = config.get_training_config()
print(training_config.pretrained_model_path)

In [20]:
from cnnClassifier.pipeline.stage_03_data_preprocessing import DataPreprocessingTrainingPipeline
from cnnClassifier.pipeline.stage_02_prepare_base_model import PrepareBaseModelTrainingPipeline


 # 1. Load training config
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    
    prepare_pipeline = PrepareBaseModelTrainingPipeline()
    updated_model, model_architecture = prepare_pipeline.main()
    print(f"🚚 Model architecture ready: {model_architecture}")
    # 2. Get data from preprocessing stage
    data_pipeline = DataPreprocessingTrainingPipeline()
    train_loader, val_loader, test_loader = data_pipeline.main()
    print(f"🚚 Data loaders ready: Train({len(train_loader.dataset)}), Val({len(val_loader.dataset)}), Test({len(test_loader.dataset)})")
    
    # 3. Initialize Training component
    trainer = TrainModel(
        model=model_architecture,
        config=training_config,
        train_loader=train_loader,
        val_loader=val_loader,
        
        
    )

    # 4. Train the model
    trainer.train()

except Exception as e:
    
    raise e

[2025-09-29 02:42:42,599: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-29 02:42:42,603: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-29 02:42:42,605: INFO: common: created directory at: artifacts]
[2025-09-29 02:42:42,608: INFO: common: created directory at: artifacts/training]
[2025-09-29 02:42:42,613: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-29 02:42:42,617: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-29 02:42:42,619: INFO: common: created directory at: artifacts]
[2025-09-29 02:42:42,621: INFO: common: created directory at: artifacts/prepare_base_model]
[Init] PrepareBaseModel initialized with model: efficientnet_b0
[Pipeline] Preparing base model...
[Step 1] Loading base model...
[Step 2] Freezing base model layers...
[Step 3] Replacing classifier head...
[Done] Base model created.
[Saved] Model saved at: artifacts\prepare_base_model\base_model.pth
[Pipeline Done] Base model p